In [1]:
import os
import time

import numpy as np
import pandas as pd
import audb
import audiofile
from scipy.io.wavfile import read as read_wav
from matplotlib import pyplot as plt
import opensmile

import sklearn 
from sklearn.model_selection import train_test_split #train/test split 
from sklearn.svm import SVC #Model
from sklearn.metrics import confusion_matrix #Metrics 
from sklearn.metrics import f1_score 
from sklearn.model_selection import cross_val_score #Cross-validation
from sklearn.model_selection import KFold

import scipy
import random #for shuffling values 
from scipy import stats

#List of prosodic features we will observe
features_prosody= ['F0semitoneFrom27.5Hz_sma3nz_amean','loudness_sma3_amean',
'mfcc1_sma3_amean',
'jitterLocal_sma3nz_amean',
'shimmerLocaldB_sma3nz_amean',
'hammarbergIndexV_sma3nz_amean']

# Function to perform one-hot encoding
def encode(label):
    if 'non-hate' in label:
        return 0
    elif 'hate' in label:
        return 1

#Gets functional features of all audios 
def functional_features(list_with_files):
    path= "data/clean_audio1/"
    
    #  Define feature extractor
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,)

    df_data= []

    for i in list_with_files:
        f= path + i
        sampling_rate, data = read_wav(f) #gets sampling rate and dimensions of data
        #print(f"Sampling rate of {f}:{sampling_rate}")
        
        db = audb.load('emodb',
        version='1.1.1',
        format='wav',
        mixdown=True,
        sampling_rate=sampling_rate,
        media='wav/03a01.*',  # load subset
        full_path=False,
        verbose=False,)

        #extract features
        signal, sampling_rate = audiofile.read(f, duration=120, always_2d=True)
        data= smile.process_signal(signal, sampling_rate)
        df_data.append(data.loc[:,features_prosody])

    df= pd.concat(df_data, ignore_index=True)
    return df
        
def t_test(dataframe):
    for i in features_prosody:
        class_0= dataframe[i][dataframe['label']==0].values
        class_1= dataframe[i][dataframe['label']== 1].values 
        t_statistic, p_value = stats.ttest_ind(class_0, class_1)
        print(f"T-statistic {i}:", t_statistic)
        print(f"P-value {i}:", p_value)
    

In [2]:
df=pd.read_csv("D:/CCiL/TFM/model/data/data_spanish_dataset.csv")

# Encoding categorical variables. Not hate speech = 0. Hate speech = 1. 
df['label'] = df['label'].apply(lambda x: encode(x))

#Storing labels in array 
labels= df['label'].values

In [3]:
#process wav files 
path="data/clean_audio1"
dir_list = os.listdir(path)
feature_data= functional_features(dir_list)

#Labels data 
feature_data["label"]=labels
df= feature_data


In [4]:
#Performs t-test 
t_test(df)

T-statistic F0semitoneFrom27.5Hz_sma3nz_amean: 2.2487934773623564
P-value F0semitoneFrom27.5Hz_sma3nz_amean: 0.026764685823525612
T-statistic loudness_sma3_amean: -2.9324582955450342
P-value loudness_sma3_amean: 0.004186531896733063
T-statistic mfcc1_sma3_amean: 2.0907844943307077
P-value mfcc1_sma3_amean: 0.039134807280088044
T-statistic jitterLocal_sma3nz_amean: -1.1565093128645185
P-value jitterLocal_sma3nz_amean: 0.2502843380505851
T-statistic shimmerLocaldB_sma3nz_amean: -3.0798712117624505
P-value shimmerLocaldB_sma3nz_amean: 0.002687346355337155
T-statistic hammarbergIndexV_sma3nz_amean: 4.132627428485812
P-value hammarbergIndexV_sma3nz_amean: 7.563214277725204e-05


In [5]:
#Train/Test split 
y_axis= df.iloc[:,[-1]]
y= y_axis.to_numpy().flatten() #Labels (class 0 and 1)
X_axis= df.drop(y_axis.columns,axis = 1)
X= X_axis.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [6]:
svc= SVC(kernel='rbf', class_weight='balanced', C=1, gamma=1e-3)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
confusion_matrix(y_test, y_pred)


array([[6, 4],
       [4, 6]], dtype=int64)

In [7]:
#Cross-validation recall

C_list= np.linspace(0.1, 10, 20)
gamma_list= np.logspace(-4, -2, 20)
recall = np.zeros((len(C_list), len(gamma_list)))


for i in range(len(C_list)):
     for j in range(len(gamma_list)):
        svc= SVC(C=C_list[i], gamma=gamma_list[j], kernel='rbf')
        svc.fit(X_train, y_train)
        y_pred= svc.predict(X_val)
        tn, fp, fn, tp = confusion_matrix(y_val, y_pred).ravel()
        recall[i, j] = float(tp+tn) / (tp + fn + fp + tn)
               

index_max = np.unravel_index(np.argmax(recall), recall.shape)

print('Max. recall:{}'.format(recall[index_max]))
C_opt = C_list[index_max[0]]
gamma_opt = gamma_list[index_max[1]]
print('Optimal C: {}'.format(C_opt))
print('Optimal gamma:{}'.format(gamma_opt))

Max. recall:0.625
Optimal C: 0.6210526315789474
Optimal gamma:0.002335721469090121


In [8]:
print(recall)

[[0.4375 0.4375 0.4375 0.4375 0.4375 0.4375 0.4375 0.4375 0.4375 0.4375
  0.4375 0.4375 0.4375 0.4375 0.4375 0.4375 0.4375 0.4375 0.4375 0.4375]
 [0.4375 0.4375 0.4375 0.4375 0.5    0.5    0.5    0.5    0.5625 0.5625
  0.5625 0.5625 0.5625 0.625  0.625  0.625  0.625  0.5625 0.5625 0.625 ]
 [0.4375 0.4375 0.4375 0.4375 0.5    0.5    0.5625 0.5625 0.5625 0.5625
  0.625  0.625  0.625  0.5625 0.5625 0.5625 0.5625 0.5    0.5    0.5   ]
 [0.5    0.5    0.5    0.5    0.5625 0.5625 0.5625 0.5625 0.625  0.625
  0.625  0.5625 0.5625 0.5625 0.5    0.5    0.5    0.5    0.5    0.5   ]
 [0.5    0.5    0.5    0.5625 0.5625 0.5625 0.5625 0.625  0.625  0.625
  0.5625 0.5625 0.5625 0.5    0.5625 0.5    0.5    0.5    0.5    0.5   ]
 [0.5    0.5    0.5625 0.5625 0.5625 0.5625 0.625  0.625  0.5625 0.5625
  0.5625 0.5625 0.5625 0.5625 0.5625 0.5    0.5    0.5    0.5    0.5625]
 [0.5    0.5625 0.5625 0.5625 0.5625 0.625  0.625  0.5625 0.5625 0.5625
  0.5625 0.5625 0.5625 0.5625 0.5    0.5    0.5    0.5    0.

In [9]:
svc= SVC(kernel='rbf', class_weight='balanced', C=C_opt, gamma=gamma_opt)
svc.fit(X_train, y_train)
y_pred = svc.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[6, 4],
       [4, 6]], dtype=int64)

In [10]:
#Validation of model (recall)

kf = KFold(n_splits=10, shuffle=True)
scores = [] 
for train_idx, val_idx in kf.split(X):
    X_train_k, y_train_k = X[train_idx], y[train_idx]
    X_val_k, y_val_k = X[val_idx], y[val_idx]
    svc= SVC(C=C_opt, gamma=gamma_opt, kernel='rbf', class_weight='balanced')
    svc.fit(X_train_k, y_train_k)
    y_pred_k= svc.predict(X_val_k)
    cm= confusion_matrix(y_val_k, y_pred_k)
    tn, fp, fn, tp = confusion_matrix(y_val_k, y_pred_k).ravel()
    recall_v = float(tp+tn) / (tp + fn + fp + tn)
    scores.append(recall_v)

print('Recall mean: {}'.format(np.mean(scores))) 
print('Recall standard deviation: {}'.format(np.std(scores)))

Recall mean: 0.68
Recall standard deviation: 0.16


In [11]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors,
)

smile.process_signal(
    signal,
    sampling_rate
)

data.loc[:,['F0semitoneFrom27.5Hz_sma3nz_amean',
 'F1frequency_sma3nz_amean',
'F2frequency_sma3nz_amean',
'F3frequency_sma3nz_amean',
'loudness_sma3_amean',
'mfcc1_sma3_amean',
'jitterLocal_sma3nz_amean',
'shimmerLocaldB_sma3nz_amean',
'hammarbergIndexV_sma3nz_amean',
'MeanVoicedSegmentLengthSec', 
'MeanUnvoicedSegmentLength']]


NameError: name 'signal' is not defined

In [ ]:
print(data.columns)

Index(['Loudness_sma3', 'alphaRatio_sma3', 'hammarbergIndex_sma3',
       'slope0-500_sma3', 'slope500-1500_sma3', 'spectralFlux_sma3',
       'mfcc1_sma3', 'mfcc2_sma3', 'mfcc3_sma3', 'mfcc4_sma3',
       'F0semitoneFrom27.5Hz_sma3nz', 'jitterLocal_sma3nz',
       'shimmerLocaldB_sma3nz', 'HNRdBACF_sma3nz', 'logRelF0-H1-H2_sma3nz',
       'logRelF0-H1-A3_sma3nz', 'F1frequency_sma3nz', 'F1bandwidth_sma3nz',
       'F1amplitudeLogRelF0_sma3nz', 'F2frequency_sma3nz',
       'F2bandwidth_sma3nz', 'F2amplitudeLogRelF0_sma3nz',
       'F3frequency_sma3nz', 'F3bandwidth_sma3nz',
       'F3amplitudeLogRelF0_sma3nz'],
      dtype='object')


In [ ]:
X= data['F0semitoneFrom27.5Hz_sma3nz_amean'].values
print(X)
# Creating histogram
#plt.hist(X, bins=30, color='skyblue', edgecolor='black')

# Adding labels and title
#plt.xlabel('Value')
#plt.ylabel('Frequency')
#plt.title('Histogram')

# Displaying the plot
#plt.show()

KeyError: 'F0semitoneFrom27.5Hz_sma3nz_amean'